# Importing necessary Modules

In [2]:
import os
import cv2
import numpy as np
import matplotlib.pyplot as plt
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Dense, Flatten, Dropout
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.utils import to_categorical
from sklearn.model_selection import train_test_split


# Data Pre-Processing

In [3]:
dataset=os.path.join(os.path.abspath(''),r"dataset")

images=[]
classes=[]


In [4]:
for dirs in os.listdir(dataset):
    imagepath=os.path.join(dataset,dirs)
    for image in os.listdir(imagepath):
        img=cv2.imread(os.path.join(imagepath,image))
        resize=cv2.resize(img,(32,32))
        img=cv2.cvtColor(resize,cv2.COLOR_BGR2GRAY)
        img=cv2.equalizeHist(img)
        img=img/255
        images.append(img)
        classes.append(dirs)

In [5]:
len(images)

10160

In [6]:
len(classes)

10160

In [7]:
X=np.array(images)
Y=np.array(classes)


In [8]:
X.shape

(10160, 32, 32)

In [9]:
Y.shape

(10160,)

# Creating Hold out Environment

In [10]:
X_train, X_test, Y_train, Y_test=train_test_split(X, Y, test_size=0.2)

In [11]:
X_train.shape

(8128, 32, 32)

In [12]:
X_test.shape

(2032, 32, 32)

In [13]:
Y_train.shape

(8128,)

# Creating model

In [14]:
outputtobeclassified = len(os.listdir(dataset))

noOfFilters=60
sizeOfFilter1=(5,5)
sizeOfFilter2=(3,3)
sizeOfPool=(2,2)
noOfNode=500
model=Sequential()
model.add((Conv2D(noOfFilters, sizeOfFilter1, input_shape=(32,32,1), activation="relu")))
model.add((Conv2D(noOfFilters, sizeOfFilter1, activation="relu")))
model.add((MaxPooling2D(pool_size=sizeOfPool)))
model.add((Conv2D(noOfFilters//2, sizeOfFilter2, activation="relu")))
model.add((Conv2D(noOfFilters//2, sizeOfFilter2, activation="relu")))
model.add((MaxPooling2D(pool_size=sizeOfPool)))
# adding dropout layer as 50 which reduces overfitting
model.add(Dropout(0.5))
model.add(Flatten())
model.add(Dense(noOfNode,activation="relu"))
model.add(Dropout(0.5)) 
model.add(Dense(outputtobeclassified,activation="softmax"))
model.compile(optimizer="adam", loss="categorical_crossentropy",metrics=['accuracy'])

In [15]:
# Reshaping X_train and X_test before passing it to imagedatagenerator(datagen) to avoid 
# "ValueError: Input to `.fit()` should have rank 4"

In [16]:
X_train=X_train.reshape(X_train.shape[0],X_train.shape[1],X_train.shape[2],1)
X_test=X_test.reshape(X_test.shape[0],X_test.shape[1],X_test.shape[2],1)

In [17]:
X_train.shape

(8128, 32, 32, 1)

In [18]:
X_test.shape

(2032, 32, 32, 1)

# One hot encoding 

In [19]:
y_train=to_categorical(Y_train, outputtobeclassified)
y_test=to_categorical(Y_test, outputtobeclassified)

# Image Augumentation

In [20]:
dataGen=ImageDataGenerator(width_shift_range=0.1,
                          height_shift_range=0.1,
                          zoom_range=0.2,
                          shear_range=0.1,
                          rotation_range=10)

# Training the  Model

In [22]:
model_training = model.fit_generator(dataGen.flow(X_train,y_train,batch_size=50),
                            steps_per_epoch=163,
                            epochs=2,
                            validation_data=(X_test,y_test),
                            shuffle=1)

Epoch 1/2
163/163 [==============================] - 44s 272ms/step - loss: 0.3751 - accuracy: 0.8750 - val_loss: 0.0683 - val_accuracy: 0.9803
Epoch 2/2
163/163 [==============================] - 44s 272ms/step - loss: 0.2515 - accuracy: 0.9216 - val_loss: 0.0410 - val_accuracy: 0.9872


# Evaluate Model

In [24]:
score=model.evaluate(X_test, y_test, verbose=0)

print("Test_score = ",score[0])
print("Test_accuracy = ",score[1])

Test_score =  0.040975507348775864
Test_accuracy =  0.9872047305107117


# Save Model

In [26]:
model.save('DIGIT-CLASSIFICATION.h5')

# Validate the Model for new data

In [27]:
from tensorflow.keras.models import load_model
testmodel=load_model('DIGIT-CLASSIFICATION.h5')

In [105]:
imOriginal=os.path.join(os.path.abspath(''),r"dataset\test\test1.png")
img=cv2.imread(imOriginal)
resize=cv2.resize(img,(32,32)) 
img=cv2.cvtColor(resize,cv2.COLOR_BGR2GRAY)
img=cv2.equalizeHist(img)
img=img/255
img=np.array(img) 


In [106]:
img.shape

(32, 32)

In [107]:
img = np.reshape(img, (1, 32, 32, 1))

In [108]:
img.shape

(1, 32, 32, 1)

In [110]:
testmodel.predict(img)

array([[6.0018818e-05, 9.9397236e-01, 1.9222109e-04, 1.4665216e-03,
        1.0216022e-03, 2.5509761e-04, 4.2779720e-06, 2.5104401e-03,
        2.1534928e-04, 3.0206295e-04]], dtype=float32)

In [122]:
predictedvalue=np.argmax(testmodel.predict(img), axis=1)

In [123]:
predictedvalue

array([1], dtype=int64)

In [116]:
# np.argmax  is used to find index of the class with highest probability

In [124]:
predictedvalue[0]

1

In [125]:
type(predictedvalue[0])

numpy.int64

In [119]:
mapping = {0:"ZERO", 1:"ONE", 2:"TWO", 3:"THREE", 4:"FOUR", 5:"FIVE", 6:"SIX", 7:"SEVEN", 8:"EIGHT", 9:"NINE"}

In [126]:
if predictedvalue[0] in mapping:
    print(mapping[a[0]])

ONE
